### **RQ1. Does support for climate policies (like carbon taxes or emissions standards) change during elections? And is this support influenced by personal or community-level perceptions of climate harm?**

To investigate this question, we estimate a **reduced Panel VAR(1)** system focusing on two dependent variables: `cc_pol_tax` (support for carbon taxes) and `cc_pol_car` (support for emissions standards). Each equation includes as predictors the **lagged values** of 17 time-varying variables, including harm perceptions, willingness to pay, policy support, political identity, and demographic covariates.

- **Harm perceptions**: `cc4_world`, `cc4_wealthUS`, `cc4_poorUS`, `cc4_comm`, `cc4_famheal`, `cc4_famecon`  
- **Willingness to pay**: `ccSolve`  
- **Climate policy support**: `cc_pol_tax`, `cc_pol_car` (included as predictors across equations to capture cross-policy influence)  
- **Political identity**: `pol_party`, `pol_ideology` — both treated as categorical variables (Republican, Democrat, Independent; Conservative, Liberal, Moderate) and dummy-coded  
- **Demographic covariates**: `dem_income_binary`, `dem_educ_binary`, `dem_age`, `dem_male_binary` — treated as ordinal or binary numeric variables

The following system of two equations is estimated:

$$
\begin{cases}
\text{cc\_pol\_tax}_t = \alpha_1 + \sum_{j=1}^{17} \beta_{1j} \cdot X_{j,t-1} + \epsilon_{1t} \\
\text{cc\_pol\_car}_t = \alpha_2 + \sum_{j=1}^{17} \beta_{2j} \cdot X_{j,t-1} + \epsilon_{2t}
\end{cases}
$$

Where:

- $ X_{j,t-1} $ denotes the value of the $ j $-th predictor at time $ t-1 $  
- $ \alpha_i $ is the intercept for equation $ i $  
- $ \beta_{ij} $ is the coefficient for the effect of predictor $ j $ on outcome $ i $  
- $ \epsilon_{it} $ is the idiosyncratic error term, clustered at the respondent level

The dependent variables, `cc_pol_tax_t` and `cc_pol_car_t`, represent support for carbon taxes and emissions standards at time $ t $, respectively. All variables are treated as **jointly endogenous**, and the model is estimated using **pooled OLS** with **clustered standard errors by respondent** (`PID`), based on 1,722 complete observations from three survey waves.

In [8]:
import pandas as pd

# Load dataset
data = pd.read_csv("../data/5-cleaned_data_3waves.csv", low_memory=False)

# Recategorize pol_party
data['party_democrat'] = data['pol_party'].apply(lambda x: 1 if x == 5 else 0)
data['party_republican'] = data['pol_party'].apply(lambda x: 1 if x == 1 else 0)
data['party_independent'] = data['pol_party'].apply(lambda x: 1 if x == 3 else 0)

# Recategorize pol_ideology
data['ideology_liberal'] = data['pol_ideology'].apply(lambda x: 1 if x >= 4 else 0)
data['ideology_conservative'] = data['pol_ideology'].apply(lambda x: 1 if x <= 2 else 0)
data['ideology_moderate'] = data['pol_ideology'].apply(lambda x: 1 if x == 3 else 0)

print("Manual dummy variables created for pol_party and pol_ideology.")

Manual dummy variables created for pol_party and pol_ideology.


In [10]:
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col

# Define continuous/ordinal predictors
continuous_predictors = [
    "cc4_world", "cc4_wealthUS", "cc4_poorUS",
    "cc4_comm", "cc4_famheal", "cc4_famecon",
    "ccSolve", "cc_pol_tax", "cc_pol_car",
    "dem_income_binary", "dem_age", "dem_educ_binary", "dem_male_binary"
]

# Dummy predictors created earlier
dummy_predictors = [
    "party_democrat", "party_republican", "party_independent",
    "ideology_liberal", "ideology_conservative", "ideology_moderate"
]

# Sort by PID and WAVE
data = data.sort_values(by=['PID', 'WAVE'])

# Create lagged versions
for var in continuous_predictors + dummy_predictors:
    data[f"{var}_lag"] = data.groupby('PID')[var].shift(1)

# Combine all lagged vars
lagged_vars = [f"{var}_lag" for var in continuous_predictors + dummy_predictors]

# Drop missing rows
panel_var_data = data.dropna(subset=lagged_vars)

# --- Model 1: cc_pol_tax_t
X1 = panel_var_data[lagged_vars]
y1 = panel_var_data["cc_pol_tax"]
X1 = sm.add_constant(X1)
model1 = sm.OLS(y1, X1).fit(cov_type='cluster', cov_kwds={'groups': panel_var_data['PID']})

# --- Model 2: cc_pol_car_t
X2 = panel_var_data[lagged_vars]
y2 = panel_var_data["cc_pol_car"]
X2 = sm.add_constant(X2)
model2 = sm.OLS(y2, X2).fit(cov_type='cluster', cov_kwds={'groups': panel_var_data['PID']})

# Output
print(summary_col(
    [model1, model2],
    stars=True,
    model_names=["cc_pol_tax_t", "cc_pol_car_t"],
    info_dict={"N": lambda x: f"{int(x.nobs)}"}
))


                          cc_pol_tax_t cc_pol_car_t
---------------------------------------------------
const                     0.5374***    0.7457***   
                          (0.1004)     (0.1076)    
cc4_world_lag             0.1884***    0.1538***   
                          (0.0305)     (0.0296)    
cc4_wealthUS_lag          -0.0454*     -0.0528**   
                          (0.0252)     (0.0220)    
cc4_poorUS_lag            0.0616**     0.0957***   
                          (0.0300)     (0.0299)    
cc4_comm_lag              -0.0232      0.0056      
                          (0.0365)     (0.0331)    
cc4_famheal_lag           0.0004       0.0027      
                          (0.0348)     (0.0305)    
cc4_famecon_lag           0.0186       -0.0181     
                          (0.0255)     (0.0229)    
ccSolve_lag               0.0916***    0.0185      
                          (0.0223)     (0.0189)    
cc_pol_tax_lag            0.3448***    0.1040***   
           

#### Panel VAR Analysis – RQ1
**Support for Carbon Taxes (`cc_pol_tax_t`)**
| Predictor                   | Coefficient | Significance | Interpretation |
|------------------------------|-------------|--------------|----------------|
| `cc_pol_tax_lag`             | **0.345**   | *** p < 0.01 | Strong temporal stability |
| `cc_pol_car_lag`             | **0.138**   | *** p < 0.01 | Support for emissions standards predicts tax support |
| `ccSolve_lag`                | **0.092**   | *** p < 0.01 | Willingness to pay increases tax support |
| `cc4_world_lag`              | **0.188**   | *** p < 0.01 | Global harm perception increases support |
| `cc4_poorUS_lag`             | **0.062**   | ** p < 0.05  | Concern for poor communities increases support |
| `cc4_wealthUS_lag`           | –0.045      | * p < 0.10   | Concern for wealthy communities slightly reduces support |
| `cc4_comm_lag`               | –0.023      | Not significant | No measurable effect from community-level harm |
| `cc4_famheal_lag`            | 0.000       | Not significant | No measurable effect from perceived family health harm |
| `cc4_famecon_lag`            | 0.019       | Not significant | No measurable effect from perceived family economic harm |
| `dem_income_binary_lag`      | 0.029       | Not significant | No strong independent effect of income |
| `dem_age_lag`                | –0.002      | Not significant | No strong independent effect of age |
| `dem_educ_binary_lag`        | **0.177**   | *** p < 0.01 | Higher education predicts higher support |
| `dem_male_binary_lag`        | 0.023       | Not significant | No strong gender effect |
| `party_democrat_lag`         | 0.086       | Not significant | Democrat ID slightly increases support, not statistically strong |
| `party_republican_lag`       | –0.266      | *** p < 0.01 | Republican ID strongly reduces support |
| `party_independent_lag`      | –0.086      | Not significant | No strong effect from Independents |
| `ideology_liberal_lag`       | **0.276**   | *** p < 0.01 | Liberal ideology predicts higher support |
| `ideology_conservative_lag`  | 0.090       | Not significant | Conservative ideology not independently significant |
| `ideology_moderate_lag`      | **0.171**   | *** p < 0.01 | Moderates show slightly increased support |

**Support for Emissions Standards (`cc_pol_car_t`)**
| Predictor                   | Coefficient | Significance | Interpretation |
|------------------------------|-------------|--------------|----------------|
| `cc_pol_car_lag`             | **0.425**   | *** p < 0.01 | Very strong temporal stability |
| `cc_pol_tax_lag`             | **0.104**   | *** p < 0.01 | Carbon tax support reinforces emissions standards support |
| `ccSolve_lag`                | 0.019       | Not significant | No strong influence from WTP |
| `cc4_world_lag`              | **0.154**   | *** p < 0.01 | Global harm perception increases support |
| `cc4_poorUS_lag`             | **0.096**   | *** p < 0.01 | Concern for poor communities increases support |
| `cc4_wealthUS_lag`           | –0.053      | ** p < 0.05 | Concern for wealthy communities slightly reduces support |
| `cc4_comm_lag`               | 0.006       | Not significant | No measurable effect from community-level harm |
| `cc4_famheal_lag`            | 0.003       | Not significant | No measurable effect from family health harm |
| `cc4_famecon_lag`            | –0.018      | Not significant | No measurable effect from family economic harm |
| `dem_income_binary_lag`      | 0.023       | Not significant | No strong income effect |
| `dem_age_lag`                | 0.001       | Not significant | No strong age effect |
| `dem_educ_binary_lag`        | 0.061       | Not significant | Education not independently significant here |
| `dem_male_binary_lag`        | 0.003       | Not significant | No strong gender effect |
| `party_democrat_lag`         | –0.060      | Not significant | Democrat ID does not predict emissions support |
| `party_republican_lag`       | –0.173      | ** p < 0.05 | Republican ID reduces support |
| `party_independent_lag`      | –0.080      | Not significant | Independents show no strong effect |
| `ideology_liberal_lag`       | **0.362**   | *** p < 0.01 | Liberal ideology predicts much higher support |
| `ideology_conservative_lag`  | **0.136**   | *** p < 0.01 | Conservative ideology predicts slightly higher support |
| `ideology_moderate_lag`      | **0.247**   | *** p < 0.01 | Moderates also show higher support |

The results show that **support for both carbon taxes and emissions standards is highly stable over time**, but also shaped by consistent attitudinal and political factors.

Key findings include:
- **Global concern** and **concern for poor communities** significantly increase support for both policies.
- **Concern for wealthy communities** slightly reduces support, possibly reflecting perceptions of inequality.
- **Willingness to pay** significantly predicts increased support for carbon taxes but not for emissions standards.
- **Political ideology** (especially liberal identification) and **party affiliation** (Republican vs Democrat) are strong predictors of climate policy support, with ideological effects generally stronger for emissions standards.

> Therefore, the answer to RQ1 is **No**:  
> There is **no sharp election-driven shift** in support for climate policies. Instead, support evolves gradually, embedded within stable ideological, demographic, and attitudinal structures.

---

### **RQ2. Does willingness to pay for climate solutions vary during elections, and what factors influence it?**
To examine how willingness to pay for climate solutions is shaped by broader climate-related attitudes, political identity, and demographic characteristics, we estimate a **reduced Panel VAR(1)** model with `ccSolve_t` (willingness to financially support climate action) as the dependent variable.

The equation includes as predictors the **lagged values** of 17 time-varying variables, grouped as follows:

- **Harm perceptions**:
  - `cc4_world`: perceived harm to the world  
  - `cc4_wealthUS`: harm to wealthy U.S. communities  
  - `cc4_poorUS`: harm to poor U.S. communities  
  - `cc4_comm`: harm to local communities  
  - `cc4_famheal`: harm to family health  
  - `cc4_famecon`: harm to family economic well-being  

- **Policy support**:
  - `cc_pol_tax`: support for carbon taxes  
  - `cc_pol_car`: support for emissions standards  

- **Political identity** (dummy-coded):
  - `pol_party`: party identification (e.g., Republican, Democrat, Independent)  
  - `pol_ideology`: ideological self-placement (e.g., Conservative, Moderate, Liberal)  

- **Demographics** (treated as ordinal or binary):
  - `dem_income`: income level  
  - `dem_age`: respondent's age  
  - `dem_educ`: education level  
  - `dem_male`: binary indicator for gender  

The full equation is:

$$
\text{ccSolve}_t = \alpha 
+ \sum_{j=1}^{6} \beta_j \cdot H_{j,t-1} 
+ \beta_7 \cdot \text{ccSolve}_{t-1} 
+ \beta_8 \cdot \text{cc\_pol\_tax}_{t-1} 
+ \beta_9 \cdot \text{cc\_pol\_car}_{t-1} 
+ \sum_{k=10}^{13} \gamma_k \cdot D_{k,t-1} 
+ \sum_{l=14}^{17} \delta_l \cdot C_{l,t-1} 
+ \epsilon_t
$$

Where:

- $ H_{j,t-1} $ are the six lagged harm perception variables  
- $ D_{k,t-1} $ are the lagged demographic covariates  
- $ C_{l,t-1} $ are the dummy-coded political identity variables  
- $ \alpha $ is the intercept term  
- $ \epsilon_t $ is the individual-level error term (clustered by PID)  

To assess how willingness to pay evolves over time and what factors influence it, we examine the `ccSolve_t` equation within a reduced Panel VAR(1) framework. The model is estimated using pooled OLS with **cluster-robust standard errors at the individual level**, based on 1,722 complete observations across three survey waves. This setup allows us to evaluate the contribution of both attitudinal and demographic factors to dynamic changes in financial support for climate action.

### **RQ2. Does willingness to pay for climate solutions vary during elections, and what factors influence it?**
To examine how willingness to pay for climate solutions is shaped by broader climate-related attitudes, political identity, and demographic characteristics, we estimate a **reduced Panel VAR(1)** model with `ccSolve_t` (willingness to financially support climate action) as the dependent variable.

The equation includes as predictors the **lagged values** of 17 time-varying variables, grouped as follows:

- **Harm perceptions**:
  - `cc4_world`: perceived harm to the world  
  - `cc4_wealthUS`: harm to wealthy U.S. communities  
  - `cc4_poorUS`: harm to poor U.S. communities  
  - `cc4_comm`: harm to local communities  
  - `cc4_famheal`: harm to family health  
  - `cc4_famecon`: harm to family economic well-being  

- **Policy support**:
  - `cc_pol_tax`: support for carbon taxes  
  - `cc_pol_car`: support for emissions standards  

- **Political identity** (dummy-coded):
  - `party_democrat`, `party_republican`, `party_independent`  
  - `ideology_liberal`, `ideology_conservative`, `ideology_moderate`

- **Demographics** (treated as ordinal or binary):
  - `dem_income_binary`: income level (low vs high)  
  - `dem_age`: respondent's age  
  - `dem_educ_binary`: education level (lower vs higher education)  
  - `dem_male_binary`: gender (male vs female/self-described)

The full equation is:

$$
\text{ccSolve}_t = \alpha 
+ \sum_{j=1}^{6} \beta_j \cdot H_{j,t-1} 
+ \beta_7 \cdot \text{ccSolve}_{t-1} 
+ \beta_8 \cdot \text{cc\_pol\_tax}_{t-1} 
+ \beta_9 \cdot \text{cc\_pol\_car}_{t-1} 
+ \sum_{k=10}^{13} \gamma_k \cdot D_{k,t-1} 
+ \sum_{l=14}^{19} \delta_l \cdot C_{l,t-1} 
+ \epsilon_t
$$

Where:

- $ H_{j,t-1} $ are the six lagged harm perception variables  
- $ D_{k,t-1} $ are the lagged demographic covariates  
- $ C_{l,t-1} $ are the dummy-coded political identity variables  
- $ \alpha $ is the intercept term  
- $ \epsilon_t $ is the individual-level error term (clustered by PID)  

To assess how willingness to pay evolves over time and what factors influence it, we examine the `ccSolve_t` equation within a reduced Panel VAR(1) framework. The model is estimated using pooled OLS with **cluster-robust standard errors at the individual level**, based on 1,722 complete observations across three survey waves. This setup allows us to evaluate the contribution of both attitudinal and demographic factors to dynamic changes in financial support for climate action.

In [11]:
import statsmodels.api as sm

# Define variables to be lagged (original + dummies + demographics)
variables = [
    "cc4_world", "cc4_wealthUS", "cc4_poorUS",
    "cc4_comm", "cc4_famheal", "cc4_famecon",
    "ccSolve", "cc_pol_tax", "cc_pol_car",
    "dem_income_binary", "dem_age", "dem_educ_binary", "dem_male_binary",
    "party_democrat", "party_republican", "party_independent",
    "ideology_liberal", "ideology_conservative", "ideology_moderate"
]

# Sort by PID and WAVE
data = data.sort_values(by=['PID', 'WAVE'])

# Create lagged versions of all predictors
for var in variables:
    data[f'{var}_lag'] = data.groupby('PID')[var].shift(1)

# Drop missing values
lagged_vars = [f'{v}_lag' for v in variables]
panel_var_data = data.dropna(subset=lagged_vars)

# Estimate the model: ccSolve_t
X = panel_var_data[lagged_vars]
y = panel_var_data['ccSolve']
X = sm.add_constant(X)
model = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': panel_var_data['PID']})

# Output results
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                ccSolve   R-squared:                       0.076
Model:                            OLS   Adj. R-squared:                  0.067
Method:                 Least Squares   F-statistic:                     362.8
Date:                Mon, 28 Apr 2025   Prob (F-statistic):               0.00
Time:                        14:43:53   Log-Likelihood:                -2346.9
No. Observations:                1722   AIC:                             4732.
Df Residuals:                    1703   BIC:                             4835.
Df Model:                          18                                         
Covariance Type:              cluster                                         
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                 

### Panel VAR Analysis – RQ2 (Willingness to Pay as Outcome)
| Predictor                   | Coefficient | Significance    | Interpretation                                       |
|------------------------------|-------------|-----------------|------------------------------------------------------|
| `ccSolve_lag`                | 0.053       | † p = 0.058     | WTP is modestly self-reinforcing over time           |
| `cc_pol_tax_lag`             | **0.110**   | *** p < 0.001   | Support for carbon taxes increases future WTP       |
| `cc4_world_lag`              | 0.063       | † p = 0.076     | Global concern may positively influence WTP         |
| `cc4_comm_lag`               | –0.054      | Not significant | No clear effect from local harm perception          |
| `cc4_famheal_lag`            | 0.042       | Not significant | No effect from family health concern                |
| `cc4_poorUS_lag`             | 0.030       | Not significant | No effect from concern for poor communities         |
| `cc4_wealthUS_lag`           | 0.009       | Not significant | No effect from concern for wealthy communities      |
| `cc4_famecon_lag`            | 0.032       | Not significant | No effect from economic concern                     |
| `cc_pol_car_lag`             | –0.033      | Not significant | Support for emissions standards does not predict WTP |
| `dem_income_binary_lag`      | 0.035       | Not significant | No strong independent effect from income            |
| `dem_age_lag`                | –0.003      | † p = 0.058     | Slight negative effect of age on WTP                |
| `dem_educ_binary_lag`        | 0.104       | Not significant | No strong independent effect of education           |
| `dem_male_binary_lag`        | 0.068       | Not significant | No strong gender effect                             |
| `party_democrat_lag`         | –0.124      | Not significant | No strong effect from Democrat affiliation          |
| `party_republican_lag`       | –0.068      | Not significant | No strong effect from Republican affiliation        |
| `party_independent_lag`      | –0.163      | ** p < 0.05     | Independents show lower WTP                         |
| `ideology_liberal_lag`       | **0.368**   | *** p < 0.001   | Liberal ideology predicts significantly higher WTP |
| `ideology_conservative_lag`  | **0.265**   | *** p < 0.001   | Conservative ideology predicts higher WTP          |
| `ideology_moderate_lag`      | **0.298**   | *** p < 0.001   | Moderates also predict higher WTP                   |

Model fit: $ R^2 = 0.076 $

Willingness to pay for climate solutions is **weakly stable** over time and appears to be primarily shaped by **prior support for carbon taxes** and **political ideology**.  
Individuals who supported carbon pricing in the previous wave were significantly more likely to express willingness to pay in the next.  
Political ideology strongly influences WTP: liberals, conservatives, and moderates all exhibit higher WTP compared to the baseline.

There is also **marginal evidence** that global climate concern (`cc4_world`) and younger age are associated with slightly higher willingness to pay.  
Other harm perception variables, concerns about local or family-level impacts, and political party affiliation **do not significantly predict** WTP after controlling for all other variables.

> So, the answer to RQ2 is **No**  
>  
> These findings suggest that **willingness to pay is primarily shaped by prior policy alignment and political ideology**, and that shifts in WTP during elections or other periods are likely driven by **deeper attitudinal and ideological structures**, not short-term emotional or political reactions.

---

### **RQ3. Does political ideology moderate the relationship between perceptions of harm and willingness to pay?**
> **RQ3:** Does political affiliation or ideology **moderate** the relationship between perceptions of harm and willingness to pay for climate solutions?

This is a **moderation question**, asking:  
- Does the effect of **perceived climate harm** on **willingness to pay (WTP)** vary depending on an individual’s **political ideology or party affiliation**?

**Moderation ≠ What Panel VAR Does Directly**  
Panel VAR models are designed to estimate **dynamic feedback** between variables over time, treating all variables as **jointly endogenous**. They are not ideally suited to test moderation, because they:
- Do not include **interaction terms** (e.g., `cc4_world × pol_ideology`)
- Do not easily accommodate **group-varying effects** within a single estimation

**What We *Can* Do Instead for RQ3**  
To test moderation, we estimate a pooled regression model with **interaction terms** between political ideology and each of the six harm perception variables. This allows us to directly examine whether the relationship between perceived harm and willingness to pay is **stronger or weaker** depending on political identity.

- **Dependent variable**: `ccSolve` (willingness to pay)
- **Harm perception predictors** (all lagged): `cc4_world`, `cc4_wealthUS`, `cc4_poorUS`, `cc4_comm`, `cc4_famheal`, `cc4_famecon`
- **Moderating variable**: `pol_ideology` (categorical or continuous)
- **Interaction terms**: e.g., `cc4_world × pol_ideology`, `cc4_comm × pol_ideology`, etc.
- **Controls**: lagged WTP (`ccSolve`), policy support (`cc_pol_tax`, `cc_pol_car`), and demographics (`dem_income`, `dem_educ`, `dem_age`, `dem_male`)

This allows us to ask:

> "Does political ideology strengthen or weaken the relationship between perceived climate harm and willingness to pay?"

**How to Do It: OLS with Interaction Terms**  
1. Include all six lagged harm perception variables as predictors  
2. Add political ideology as a main effect  
3. Add interaction terms between each harm perception variable and political ideology  
4. Control for lagged WTP, policy support, and demographic characteristics

What This Tells Us  
- If interaction terms are significant, it suggests that **the effect of perceived harm on WTP varies by political ideology**
- A positive interaction means harm perception increases WTP more strongly among liberals; a negative interaction suggests the opposite

To assess whether political ideology moderates the relationship between climate harm perceptions and willingness to pay (WTP) for climate solutions, I estimate a pooled OLS model with interaction terms, where `ccSolve_t` (willingness to pay at time $ t $) is regressed on:  
- the six lagged harm perception variables  
- political ideology  
- the six interaction terms  
- lagged WTP and policy support  
- demographic controls

In [3]:
import pandas as pd
import statsmodels.api as sm

# Define harm perception variables
harm_vars = [
    "cc4_world", "cc4_wealthUS", "cc4_poorUS",
    "cc4_comm", "cc4_famheal", "cc4_famecon"
]

# Create ideology groups
data['ideology_group'] = data['pol_ideology'].apply(
    lambda x: 'liberal' if x >= 4 else ('conservative' if x <= 2 else 'moderate')
)

# Run models for each ideological group
for group in ['conservative', 'liberal']:
    subgroup = data[data['ideology_group'] == group].copy()
    subgroup = subgroup.sort_values(by=['PID', 'wave'])

    # Create lagged variables
    subgroup['ccSolve_lag'] = subgroup.groupby('PID')['ccSolve'].shift(1)
    for var in harm_vars:
        subgroup[f"{var}_lag"] = subgroup.groupby('PID')[var].shift(1)

    # List of lagged predictors
    lagged_vars = [f"{var}_lag" for var in harm_vars] + ['ccSolve_lag']

    # Drop missing values
    subgroup = subgroup.dropna(subset=lagged_vars)

    # Estimate model: ccSolve_t ~ all harm lags + ccSolve_lag
    X = subgroup[lagged_vars]
    y = subgroup['ccSolve']
    X = sm.add_constant(X)

    model = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': subgroup['PID']})

    print(f"\nSubgroup: {group.upper()}")
    print(model.summary())


Subgroup: CONSERVATIVE
                            OLS Regression Results                            
Dep. Variable:                ccSolve   R-squared:                       0.082
Model:                            OLS   Adj. R-squared:                  0.069
Method:                 Least Squares   F-statistic:                     6.589
Date:                Tue, 15 Apr 2025   Prob (F-statistic):           3.31e-07
Time:                        09:21:15   Log-Likelihood:                -598.01
No. Observations:                 516   AIC:                             1212.
Df Residuals:                     508   BIC:                             1246.
Df Model:                           7                                         
Covariance Type:              cluster                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const           

- The six lagged harm perception variables:
  - `cc4_world`, `cc4_wealthUS`, `cc4_poorUS`, `cc4_comm`, `cc4_famheal`, `cc4_famecon`
- A lagged dependent variable: `ccSolve_lag`

Political ideology was used to define subgroups:
- **Conservatives**: `pol_ideology` ≤ 2
- **Liberals**: `pol_ideology` ≥ 4

Standard errors were clustered by individual (`PID`). This allows us to test whether the relationship between perceived harm and WTP varies across ideological groups.

**Conservative Respondents**  
| Predictor         | Coefficient | Significance | Interpretation |
|-------------------|-------------|--------------|----------------|
| `cc4_world_lag`    | **+0.131**  | ** p < 0.05 | Global harm perception increases WTP |
| All other harm variables | Not significant | — | No measurable effect |
| `ccSolve_lag`      | +0.073      | Not significant | Weak self-reinforcement |
| $ R^2 $          | 0.082       | — | Moderate explanatory power |

→ Among conservatives, **global harm perception significantly increases willingness to pay**, while other forms of perceived harm do not. This suggests that **issue-based concern**, when framed globally, can motivate financial climate commitment in this group.

**Liberal Respondents**  
| Predictor         | Coefficient | Significance | Interpretation |
|-------------------|-------------|--------------|----------------|
| `ccSolve_lag`      | **+0.151**  | ** p < 0.01 | Strong temporal stability in WTP |
| All harm variables | Not significant | — | No predictive effect on WTP |
| $ R^2 $          | 0.029       | — | Lower explanatory power overall |

→ Among liberals, **none of the harm perception variables significantly predict willingness to pay**. Instead, WTP is driven mainly by its **own past values**, indicating that climate commitment in this group is **more stable and ideologically anchored**, rather than responsive to short-term shifts in perceived threat.

These findings suggest that **political ideology moderates the effect of harm perception on willingness to pay**:

- Among **conservatives**, WTP is more responsive to perceived harm — particularly at the global level — indicating that concern about broad climate risks may motivate engagement among right-leaning individuals.
- Among **liberals**, WTP appears to be **less dependent on perceived harm**, and more internally consistent over time, likely reflecting a pre-existing ideological commitment to climate action.

> So, the answer to RQ4 is **Yes**
> 
> This supports the broader theoretical claim that **liberals and conservatives process climate-related risks differently**, with conservatives requiring stronger threat-based cues to express support, and liberals maintaining support based on pre-existing values.

